In [ ]:
#TODO:
#1) Use the Track Attribute API for tempo, loudness, camelot, duration/1000 (seems to be a problem wit Spotify's servers. It struggles to retrieve information for all songs...crashed after 30 songs)

In [19]:
import pandas as pd
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [20]:
cid = "2db050f400f245738228ef384306044b"
secret = "6309c1f0455b43428d2cf142e4130a64"

client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [90]:
songs = pd.read_pickle("songsWithSpotifyID.pkl")

In [48]:
songs["Tempo"] = None
songs["Loudness"] = None
songs["Key"] = None
songs["isExplicit"] = None
songs["Duration"] = None

In [24]:
#getExplicit function that takes in a songID and returns if the song is explicit or not
def getExplicit(songID):
    return sp.track(songID)['explicit']

In [69]:
#getTrackAttributes function that takes in a songID and returns song attributes such as, Tempo, Loudness, Key and Duration
def getTrackAttributes(songID):
    return sp.audio_features(tracks=songID)[0]

In [74]:
#getSongKey function that takes in a pitch class and returns the key. Reference: https://en.wikipedia.org/wiki/Pitch_class
def getSongKey(pitchClass):
    if pitchClass == 0:
        return 'C'
    elif pitchClass == 1:
        return 'C#, Db'
    elif pitchClass == 2:
        return 'D'
    elif pitchClass == 3:
        return 'D#, Eb'
    elif pitchClass == 4:
        return 'E'
    elif pitchClass == 5:
        return 'F'
    elif pitchClass == 6:
        return 'F#, Gb'
    elif pitchClass == 7:
        return 'G'
    elif pitchClass == 8:
        return 'G#, Ab'
    elif pitchClass == 9:
        return 'A'
    elif pitchClass == 10:
        return 'A# Bb'
    elif pitchClass == 11:
        return 'B'
    else: return None

In [49]:
for index, row in songs.iterrows():
    #get explicit attribute from Get Track API
    print(row['SpotifyID'])
    explicitValue = getExplicit(row['SpotifyID'])
    attributes = getTrackAttributes(row['SpotifyID'])
    
    #filling in values
    #     print(attributes['tempo'], attributes['loudness'], attributes['key'], attributes['duration_ms'])
    songs.isExplicit.iloc[index] = explicitValue
    songs.Tempo.iloc[index] = attributes['tempo']
    songs.Loudness.iloc[index] = attributes['loudness']
    songs.Duration.iloc[index] = attributes['duration_ms']/1000
    songs.Key.iloc[index] = getSongKey(attributes['key'])

6VzcQuzTNTMFnJ6rBSaLH9 False
7sWRlDoTDX8geTR8zzr2vt False
1dGr1c8CrMLDpV6mPbImSI False
47otoIkhx3fkdivEXL5OB6 False
273TiTHLlHSRe5zrzs7wvD False
6YdQgWSpsxhVeX6Xmv3IFJ True
0csmcZqM3uTKfjxehw0Lw2 True
5fclVBnzaGrbucvVMC228o False
3e9HZxeyfWwjeyPAMmWSSQ True
02Ey48n33H3erWJRZAtTFj False
6ZOXF7nbzCpPVNlC0AG1rF True
2kAIpGWnlFLQh48iut6Zzq False
4j9joWIC7UjtTz3zpBKWhB False
2OKo7g3KfmCt3kyLvUAL0g False
312WNtMs3F28cUukaPY9bo False
03eJ2DclFWXYU8GWgANdmZ True
7azo4rpSUh8nXgtonC6Pkq False
178OI1A3qjROeFeh8lmNwW False
39zWYYZStDgWi32sOU9AX4 False
08mG3Y1vljYA6bvDt4Wqkj False
3UUPQ4sAtAVnkcaSr06NBi False
487OPlneJNni3NWC8SYqhW False
124NFj84ppZ5pAxTuVQYCQ True
03d8S7FBY2kIzhP0E7l4rn False
0pMfPzdRiPAoSM9yWF4OcT False
4KRNWJZb8dgm7YqaHpsMyf False
2U9kDk5mlHYunC7PvbZ8KX False
4so0Wek9Ig1p6CRCHuINwW False
5DTOk0tCrmicVQP4mc6hkr True
30DnQCN64v8xBpGZpLgb6l False
1ssFg8Xr5A8O2U6JkKjZRz False
0h9fnCSnbUgOEgibnQByFv False
30FURVTCpbKyykjSEQzGkH True
7MjSipTto9QljYzZnloXOn True
7MXVkk9YMctZqd1Srtv4MB 

6uYgCE2OEulOw5zhHvZsHf True
6Dqja51kHdm93TJGkX7T16 True
07NMFSwppnky8JyIQIxAQZ True
7tgjDlQsMR8RvkdlNCJA58 True
3Z9vHQOOPvRtOnTPyXLkWD True
4hQ6o8ECY5MyHyodnyOHEw True
1FxTz4x5X7ewAICkKhB3kA True
01kOu8BA8yzAHx9LUidTi9 True
56S5YTNqlsXSnR5T1S2oyJ True
1W24W6jQegnNh0x5DfBBPT True
3Kd6Wp3lG5HXMB2dlvuy7y True
5e4oAwSsIzkNZxh4fLSKUH True
2UUnhLf51JSrji1FAbJ17G True
39WVsfrPyaLCx04khGj3fN True
2cSqeOJIaFPEF7N5J7xiIZ True
1M4pM5XgwxasguJP1dJIC6 True
55L35RJDZHEd5kIqz0Umkv True
6v36HjllB4mRIFlT3U1ZPL True
5w7wuzMzsDer96KqxafeRK False
41zXlQxzTi6cGAjpOXyLYH False
1RGasjWLZ4qMN7wbtkLa3u False
7eB1V5LvAdxCc7brfGhRRo False
3WxmlTZ85sCYFnuIXmUAEe False
1ni8ZTAY1GHXEFOGHl7fdg False
7hDVYcQq6MxkdJGweuCtl9 False
1WsEgieHsWWndAzLkmV105 False
6X7qcg95CAeJpTjtg1wooM False
0QVvkAOstRG26pVYN95CeL True
6vWzkhoAp1SF4PEJ0iLKYQ True
3eeyKNIBrt5TH4y0y3QkOu True
0eGvris9sYZMOsTMHT20Td True
4vSoyDfL0uZyUoWu5NNNsQ True
6klLvorLoo1sxvZcjJIY8I True
5A6tFAdihqILHJrWuR6wD4 True
2m9axSosHP92250FvnnobM True
0qmWrPu7fiZ

1lUYap3IVPOLZTAq8828F6 False
2IBrLjMW54GQsit7s2GJaY False
1lOe9qE0vR9zwWQAOk6CoO True
6eSU1exzDvBEXmBRxspeh5 True
1kBgQ2cBNONDo7ABrvmRfu True
4guBZjUyrGoHsTahO3IHdj True
1ipyiYqvds0wcaFDTW6N4X True
3gOh1UDmnGWN4wrEcaulri True
4ESyocZUQ3fOlCbNP6n17i True
2BULAwmRpScmZE7YOF07e5 True
4QjLUh87xXSWoOVFbtjkWb True
7uVu5amg3zP3p2n22WhY2z True
10mIc2UoZkg1kaOfKSZ8y1 True
4Bhdiu8zhwQR3tXy8YMbCj True
4e0FYxSROat25pH16zMTZI True
2SFaHGfSeOYBA0foNEVlIY True
4ndxiJH8MkwIvPpRn8DVJA True
06xih8BhxlC9EbjqHWBtEs True
7FITP8MVBwdQcbNAr0aVio True
2WsuSYJNXGKXVYkHPnq2yp True
6B4KWBeFdHrpVezjTqggWr True
2gwkD6igEhQbDQegRCcdoB True
3WyRgi8CzQnhzO0xw79tTS True
0SsTkfOhM7e87trwjEqBQN True
3yXP1unIs1X9U5FbEarv7e True
7f1zjZG77S891Uv4O68yfk True
7uxbabRF4omY1MhmIgaZ6Y True
6uIpf2e5PrB0ZMufi0AWpL True
1N7CHs4eGjgr3gjyUgq75Y True
2gjPbUsttk6GqJOUAh9WzN True
5U8sc8et1PiuDQfIQ2J98a True
2ky7vgodn1YedpeS00lUqt True
6aYi4lYcAohNhlNWsbWfy0 True
4uu7gKd1PffC7QEMcMk0Ro True
5qDsijQM75KTdv3RtSPx91 True
0RLLvYBopDwooCyvnT

0fB77VOZ2FkQeKLv1DuEwp True
1ghvzmzpx2nnrbx7wtpMgo True
7k51pIqTGEHHZOnpI11Qq2 True
1TMWcbxL5YF8rKsFHv5hAP True
1L0z2CQKrKJ2spYSMkfim9 True
3Dv1eDb0MEgF93GpLXlucZ True
4EivmOT13NMpNSfTKn9p4s True
1j6NZFHrpc09YmEZuSlmwf True
2OAcH9SD8ehxuG0tWNe0cU True
60ynsPSSKe6O3sfwRnIBRf True
5xIDCYGTyLaDkiaaulVPcJ True
1ZiJGdjNtvgngdT3wRKVqS True
7f5trao56t7sB7f14QDTmp True
7p2ewixAShLpjDZrnzZK7c True
2qcG0nZ6S3zZV0UrkY5nFo True
17Yq72h0p15OhCbZ5lJ5gd True
6KBYefIoo7KydImq1uUQlL True
2Yl4OmDby9iitgNWZPwxkd True
00B7TZ0Xawar6NZ00JFomN True
58q2HKrzhC3ozto2nDdN4z True
5DfWF0jqMgZDCiu9zJENTA True
1vBeIXlzbAieGoDqInav5j True
0dIoGTQXDh1wVnhIiSyYEa True
0CfmNX2TXk7yoJqHJiixnp True
74cVJfmK18rhWnscIpGXjt True
1f5PNhkNgUpvDEeZfcIlO1 True
6SIV02mskzzc3KXK7t4NHj False
2BS1mU9kuKaPTKjXsmM42B False
3QV4Ywr4QOSnwsFAgfJ8bt False
085I8E5w4vB70EOW2L8XgY False
250UTGbkM9fIGNfapGA58b False
76JhQ47ZGWLpPwSLjeYBB5 False
5SdMXQmfV4OZx5O05986d3 False
1XvAQXFRKacD3jp9uxarhZ False
51QRwQSBy7HNew2TY6MATT False
72STcTzwvQH

1y3bE5i57eUm3hfLAc7h08 False
3zT2616WcD9Df5m21OsCX0 False
0FfX0nofboxZjP0diT1eFv False
2SJf7W18D1AEpr8L9ATu0c False
1dekgAFF9uTCqLsklDaCWb False
1zB4vmk8tFRmM9UULNzbLB False
2Iug43iQrHN8CbGsUd2tEt False
3adVVrcE2KS26OL4rYd27O False
278kSqsZIiYp8p3QjYAqa8 True
5jGUJXnELDaDTgiRjD9lsg True
6kN06eEOblRAuWI0xUUS91 True
6it15CsDlkqB7N4lF0C1qM True
5VQpJ2AY2iMzrRZKgcvNjh True
4vCAzANUWDE24URV6wQ4ra True
4UEuIEv9Wc3wtiWUplGJ7q True
35wvL50xvKpCHEJPxLOLPI True
4uP3pUP18ennmz0tb3GphR True
5mj8WVFcKdGA8p9HWGTSLc True
1khmgu0pveJbkbpbkyvcQv True
69ZaPBHhRMRDjRpW1ivnOU False
6pZHZndlo57dPCYnvlYFOE True
2OWVCFTolecLiGZPquvWvT False
3gE4eQH3K83Sght0ZLvuBK False
48YkpE5enOln5c7jXSTuHL False
3sYWyVGfPuit80szWXhrRA False
7oBazz4xCO7Ul3MJHhKy0Q False
15VRO9CQwMpbqUYA7e6Hwg False
6DX6W2QXiVlKYQLrQN8y83 False
725QAOexNLmGZiFMN7U8pF False
0xHch89AN9RlN8Nm4kJzs3 False
2RyiGdY33PR4aCTUolEhk7 False
1tlTFxB34VorrZnJNfRWcs False
2smax1OWAxOo63p14G8KZx False
2vfCSwToh0Am0O4zgHXfn7 False
4UJmPSJsBsIR1U0N79BU1g Fal

...

1UcV9ClAFKKkoagbOkggZs False
5eljScRJp5U3zxQO0xGFV7 False
3Jec6BQxWfoPIZht3ibQWo False
4buoMJKxbFAw0UCZtwq4pK False
2BgZrbbAZxSnyE7U5efVrG False
1ZpFxOhpzT1MDCq7zjtOV1 False
1OqkHhYOcqfwYx7LqlAE8Y False
5oXdWFCLkBX5XmGbTrlK40 False
2SytGgja3g2R2N9lzV6n3E False
2Nib4r0saYQf2pUV4RFDhl False
3bkkMZEAhx7rTVz1C0itRQ False
0ono6UCNVZ1XqOm6j78Blu False
4vTgx6h4seHvkuFh84JXYP False
3op7HNwLli54MBjFGzIlZO False
2QyuXBcV1LJ2rq01KhreMF False
1aKuG1GG76xPLUkjBbQo3W False
4AlihYDqxXshKhvh5tnMfP False
4BKOjYosPhw334moS3wlbO False
7w05lsU2VLSeiT8Nukrv5M False
1xfXXujIlLZAx5wpu46Nyk False
37TSJmYKLxXmx8OehFpqMN False
44WLOqH7QayQOQdeUHeKUK False
3r9FFjLrAIQjoR8pSHVPC9 False
3IB5qOeMayvpOdHxYCL5tZ False
5YEAvnL3klXUE7etBXvMlp True
7MYFKS7XpVz1JQDK38pr8N True
1aMokXl3uKDvCh4BBcjFOA True
3qHgGyJY4GpXNOK4WL4NSo True
0KVEBQA98JvkmB5ZydQ13k True
37LWyqxXEUrjMxrhPlAiNF True
3ug164pc9RGmq4TM6x61vJ True
4RhRDVXc3xFojyyjyh1jmN True
7HjRPlzOwN72D8gjn9zURA True
6fEdWrEYspdneQMNIT33I4 True
1m5ThpiPCchWtaBIAnNfz8 T

0yOerfNZxaBB0YsabBbSC5 True
6vR7ch2RfzdmmOuDZr4187 True
2eHy0Irk6LKSqmlnOFkckF True
2ipFNBdVocxiHjoQezbmCI True
3XgeoD9wwkfATGe4d2zfSL True
0J5PWxUI8UQbPT5ldFbc47 True
7fEoXCZTZFosUFvFQg1BmW True
1GieuvkMLaSeu57s1BRVNP True
3PjSkZGM7rpNPymaesfZte True
6to1tNRE8t1WuTSdvDnsmw True
5v4OvUHVi7LxQxIXzTPNww True
6KwMXFvpVOlQQRsY47b3ec True
3JjDDzUBkbBE6QTmlntWFX True
02XlaBr5k34zLI1lG6uCSP True
2WRn4AHv6hXK7914MExhze True
0ntNhZSeeCTflZWIHharn1 True
5UgbeEHYOEy8XLQER9Rt7Z True
1gTOosCIBHaJ6sPwJ3Pqdh True
1YVhP58OOuJKDcwM7aO8K1 True
1K3k4wMCpgDQLf8OCSHnnz True
38OkxYU9CQb3bw6g2mOD6r True
4ICJf5NgrMs2pOTUTALX0A False
6vsBCrM20zIKYkjWQ2qJAo False
7126kIzOFbWsvWcz6s4ITQ False
2sE8dFhRTmNKPn398XcrmX False
1qM4vhLgS0XxywxoHV05lC False
3vX6vj635aCs3WnzOKopo6 False
5WERUlMEyhQfG9bwS34PF3 False
4vfhcnJ0h2goTTD6H6LOaf False
1u5I9nhzlSB9wa1szYgjAM False
1zAvqKtXTFVoMH58U6PLBr False
2FVMbLuIxo0dz3KLXgN6Jh True
3iLqD9D7hnWVhlizXrZQWt True
3jNRtxAd3m3NYvnCPi88pe True
5LdFdNBPGIGIAaJFMrsAYe True
3MG3qM6Z32

2HhJ0b1AwX5Vgcnf3zzx1p True
6aCBFAqcLu11o8wmT6q5De True
6lDfHLmWTr1JnVgQhVBy41 False
6RGjH8nWiFvVS3FZUTzvRq True
6YapMTu34xuchs2ggp0k3Q True
0kqTPWd8WgI7wmVuspmjXz True
3bV8YN5Cl2yHIx79zI5tSy True
0U84VaJsucHDdJyZXFOW8V True
3q0U6eqmHObU2cy0O6VHGS True
6I2Z2pUjQs1FTUcg1nDthZ False
0vCuS7ckBat4b3jQOGyEOq False
2zU7F0FcXfW1zv7Qkq4nom False
5QWyw2PxLy8iKcsERynEev False
5VB1uSUXMIGHcnZ2Y4jLoA False
4YgoHd5tUQMgRQcYG0Vv0i False
2A0dGdkA6sOUj8FKH6fJ0D False
4DaPWHZchEJuGKNkvM4wLC False
2LbD4j2qZAi2RQpyqcW9P2 False
4rkgfDNnBXrqD69qbL0G7d False
1FJZABpb7oPxreaPE7ncyI False
72c9An3Uf1iUhJVO4JUVzu False
4UI97tlERi8pV9cVnWb8ZZ False
2SbOhIijt3eHLueYsPpqzM False
7xWb5uQj7CPVhlfJNnf1lj False
3jwDLkZHaEjCSi02JaXPlK False
0WFJEeyfg5iSaxCPMcVNRi False
4dqEzdFAYkj1rjYLVDuTlN False
5nSlWvm37WSY2Nt2x76rqn False
3I3EYqQfq7WRf4LqvuRb0n False
5txy3GxBkbppwgk7TrFtuP True
0gRbbSuaaELTf26HXRByQH True
3nZJsFpBt4KglQMs6jmejR True
78aTHrV8AZkRX6DK5rQioH True
4p2UhyfUgfyL52rmvxAR0M True
6foJWmFEEuUsuDSMFPznma True

72j2I6r1qNbwzVkicUAFYx False
07q5QpbAuZqVR2NqKLDxw0 False
6kMFhHABG3rqIQ7uCqQswP False
0jREQOZ2lxTNdWzLujpT0i False
5Q42MkhwFsuM8uk8wkHTR7 False
3AU8zz7ptAUdNzdNDruWGk False
7K0tzjli9Ksqy4R2YuJriV False
3Vf19D3mddJCj1JKonqwuk False
7e4yf2ICJQqfuiKyk3oaoC False
5V9WduK9uYEz53AletGalG False
5ql94CS36c6PM7KXTeSqTh False
1ahRPMw2Cpv1HfUNW8Ykw4 True
1Yt5ws5j7JENlip4XbLZgy True
1fPnUpoXCpD2NDI4vPVeLw True
7DHNcoR73yUYCSn6PqBgkH True
5LowZZg0kBpXRLLEY0sO3Q True
26nWw7LWlId9myIxHMHXmm True
4xqIYGwwZTEem9U8A42SPF True
0COXNxwxkWeA4hInT7Zomk True
1ptXPpqYdJNYu4g7YEZ303 True
7uDTc1eJwsD7iMZLhdp8LH True
7n0lXKEOaFxrAU0R93fIQh True
0nh0CYu2oWeHr10rwgweTc True
2h5wkA3keIclvfAJ95VMAo True
0CnE69w3UHMTH3r5LGbAjP True
16q7iofxCkvWBKef4fMKf3 True
1GT4i5dtm89tSc8eaidpNf True
5ruzrDWcT0vuJIOMW7gMnW False
2t0wwvR15fc3K1ey8OiOaN False
2mDYYGaGd9uXKkK2YhDA3i False
55N8cxpE1QDoeaaNqUnoZ2 False
6IRzBP4gVoV4D2zHmocoWy False
7lu6YLrlB1MU7HW8DJSuPa False
1aOxOpH4AkGAd8OMrKjyNY False
5ncofiedz9Ej6YkvvXMKVn False
1

6yg9sPJsnGStmIIcxzOaTL False
15VDHAMS4JCLziCjNkmQl1 False
3ngPR5K3ZCnLxAuSDac7V6 False
0u4dw7TujMZivFm9FyaIfu False
7ilkiH0aHXoJykJace22cH False
3lY1DbeIc84Y7kDCncyWbs False
45KIvymezcVSStboODBQWX False
69jstgacdd40OwG9cLl45r False
18C0OMcQh37EsWMCzMaget True
40K07WyXb7gJSB2bgbJPBC True
2QTlfHvMFV4j1e1uw89Ioh True
3ViyRR4HmK0LrFNazQjuiz True
7rd0oaQvVOS1Q52QXVJIRM True
3cPlcQrg0l9UTbjnwRAQXB True
5NCxEMhvyvWjvmBenrJV3d True
1kA3DTGCwnnp8SUYkk6kuC True
2F1EJARygYXDXYsQB1LOgz True
39DIUoM4SnOUnjvcXfNlQh True
53gRCOri32jYZuL3LWd8Wt True
78I1aQPoGHXLy56MQJDHL5 True
12k9y4Rw0nvv44VqLFxiHi True
3CySa5lS8wy7pRHCYCKlCR True
6XaQk2aBCvpjzRb52sL1Wu True
7gjDKOGSLxVLpUsz7yiinw True
2vei2yZVAuJMWcrASjlhkR True
08Nxk0ei6Gem3OehA9P27d False
7n9Dp8yURvNGTyAMXyfmb1 False
1kLlHxLKzwTWgFUnwNFbz1 False
42C7XkZNUA4ZHjsI1LwNrH False
48oIVM8IPmPpKlzJ0wzl9j True
6GnunkdHwcI6cZCKe2wt3x False
1w0YVbR3A6QHxKtotMctko False
0jevq633Havix9k217P3F5 False
7LqXxG6IXTL7KbiywNZ5Tn False
6SNrDgDlYsSLLI9vEQy4vS False
7Fx

In [ ]:
# songs.reset_index(inplace = True)
# songs.drop(columns = ['index'], inplace = True)
# songs.head()
# songs.to_pickle('songsWithSpotifyID.pkl')
# songs = pd.read_pickle('songsWithSpotifyID.pkl')